# Classify chemicals using ClassyFire and NPClassifier

### Import modules

In [1]:
from collections import defaultdict
import time
import json
import requests

from pandas import json_normalize

from pybatchclassyfire import *
import pandas as pd
from tqdm import tqdm

from rdkit import RDLogger  
from rdkit.Chem.inchi import MolToInchi, InchiToInchiKey
from rdkit.Chem import MolFromSmiles

/Users/danieldomingo/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
INFO:rdkit:Enabling RDKit 2022.09.1 jupyter extensions


In [2]:
%config Application.log_level='INFO'
import logging
logging.getLogger('s3fs').setLevel(logging.INFO)
logging.getLogger('botocore').setLevel(logging.INFO)
logging.getLogger('fsspec').setLevel(logging.INFO)

In [3]:
RDLogger.DisableLog('rdApp.*')

In [4]:
DATA_DIR = '../data/'

Load newmann data

In [5]:
natural_products_df = pd.read_csv(
    's3://enveda-data-kg/others/cdd/newman_natural_products.tsv',
    sep='\t',
    usecols=[
        'curated_name',
        'pubchem_name',
        'pubchem_id',
        'smiles'
    ]
)

synthetics_df = pd.read_csv(
    's3://enveda-data-kg/others/cdd/newman_synthetics.tsv',
    sep='\t',
    usecols=[
        'curated_name',
        'pubchem_name',
        'pubchem_id',
        'smiles'
    ]
)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): enveda-data-kg.s3.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://enveda-data-kg.s3.amazonaws.com:443 "HEAD /others/cdd/newman_natural_products.tsv HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://enveda-data-kg.s3.amazonaws.com:443 "GET /others/cdd/newman_natural_products.tsv HTTP/1.1" 206 968925
DEBUG:urllib3.connectionpool:https://enveda-data-kg.s3.amazonaws.com:443 "HEAD /others/cdd/newman_synthetics.tsv HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://enveda-data-kg.s3.amazonaws.com:443 "GET /others/cdd/newman_synthetics.tsv HTTP/1.1" 206 2174410


In [6]:
natural_products_df.shape, synthetics_df.shape

((398, 4), (935, 4))

In [7]:
synthetics_df.head(2)

,curated_name,pubchem_name,pubchem_id,smiles
0,alizapride,6-methoxy-N-[(1-prop-2-enylpyrrolidin-2-yl)met...,135413504,COC1=CC2=C(C=C1C(=O)NCC3CCCN3CC=C)NN=N2
1,amezinium methylsulfate,6-methoxy-1-phenylpyridazin-1-ium-4-amine;meth...,71926,COC1=[N+](N=CC(=C1)N)C2=CC=CC=C2.COS(=O)(=O)[O-]


In [8]:
natural_products_df.head(2)

,curated_name,pubchem_name,pubchem_id,smiles
0,aclarubicin,"methyl (1R,2R,4S)-4-[(2R,4S,5S,6S)-4-(dimethyl...",451415,CC[C@]1(C[C@@H](C2=C(C3=C(C=C2[C@H]1C(=O)OC)C(...
1,Netilmicin sulfate,"(2R,3R,4R,5R)-2-[(1S,2S,3R,4S,6R)-4-amino-3-[[...",62115,CCN[C@@H]1C[C@@H]([C@H]([C@@H]([C@H]1O[C@@H]2[...


In [9]:
smiles_in_data = set(natural_products_df.smiles.unique()).union(
    set(synthetics_df.smiles.unique())
)
len(smiles_in_data)

1314

In [10]:
all_inchikeys = set()
skipped = 0

for smiles in tqdm(smiles_in_data):
    try:
        mol = MolFromSmiles(smiles)
    except:
        skipped += 1
    
    all_inchikeys.add(InchiToInchiKey(MolToInchi(mol)))

100%|██████████| 1314/1314 [00:01<00:00, 985.77it/s] 


In [11]:
len(all_inchikeys)

1314

NP Classifier

In [12]:
np_classifier_rows = []

skipped = 0

for smiles in tqdm(smiles_in_data):
    
    response = requests.get(f"https://npclassifier.ucsd.edu/classify?smiles={smiles}")

    json_data = response.json() if response and response.status_code == 200 else None
    
    if json_data:
        np_classifier_rows.append({
            'smiles': smiles,
            'class_results': json_data['class_results'],
            'superclass_results': json_data['superclass_results'],
            'pathway_results': json_data['pathway_results'],
        })
        
    else:
        skipped += 1

np_classifier_df = pd.DataFrame(np_classifier_rows)

  0%|          | 0/1314 [00:00<?, ?it/s]DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): npclassifier.ucsd.edu:443
DEBUG:urllib3.connectionpool:https://npclassifier.ucsd.edu:443 "GET /classify?smiles=COC1=CC=C(C=C1)CCN2CCC(CC2)NC3=NC4=CC=CC=C4N3CC5=CC=C(C=C5)F HTTP/1.1" 200 335
  3%|▎         | 36/1314 [00:06<03:05,  6.88it/s]DEBUG:urllib3.connectionpool:https://npclassifier.ucsd.edu:443 "GET /classify?smiles=CC%5BC@H%5D(C)%5BC@@H%5D(C(=O)N%5BC@@H%5D(CCC(=O)O)C(=O)N%5BC@@H%5D(CC1=CNC2=CC=CC=C21)C(=O)N%5BC@@H%5D(CC(C)C)C(=O)N%5BC@@H%5D(CCCCN)C(=O)N%5BC@@H%5D(CC(=O)N)C(=O)NCC(=O)NCC(=O)N3CCC%5BC@H%5D3C(=O)N%5BC@@H%5D(CO)C(=O)N%5BC@@H%5D(CO)C(=O)NCC(=O)N%5BC@@H%5D(C)C(=O)N4CCC%5BC@H%5D4C(=O)N5CCC%5BC@H%5D5C(=O)N6CCC%5BC@H%5D6C(=O)N%5BC@@H%5D(CO)C(=O)N)NC(=O)%5BC@H%5D(CC7=CC=CC=C7)NC(=O)%5BC@H%5D(CC(C)C)NC(=O)%5BC@H%5D(CCCNC(=N)N)NC(=O)%5BC@H%5D(C(C)C)NC(=O)%5BC@H%5D(C)NC(=O)%5BC@H%5D(CCC(=O)O)NC(=O)%5BC@H%5D(CCC(=O)O)NC(=O)%5BC@H%5D(CCC(=O)O)NC(=O)%5BC@H%5D(CCSC)NC(=O)%5BC@

  6%|▌         | 73/1314 [00:13<03:45,  5.51it/s]DEBUG:urllib3.connectionpool:https://npclassifier.ucsd.edu:443 "GET /classify?smiles=CC(C)C%5BC@@H%5D(C(=O)N%5BC@@H%5D(CCCN=C(N)N)C(=O)N1CCC%5BC@H%5D1C(=O)NCC(=O)N)NC(=O)%5BC@@H%5D(CC2=CNC3=CC=CC=C32)NC(=O)%5BC@H%5D(CC4=CC=C(C=C4)O)NC(=O)%5BC@H%5D(CO)NC(=O)%5BC@H%5D(CC5=CNC6=CC=CC=C65)NC(=O)%5BC@H%5D(CC7=CN=CN7)NC(=O)%5BC@@H%5D8CCC(=O)N8 HTTP/1.1" 200 487


  8%|▊         | 109/1314 [00:19<02:49,  7.11it/s]DEBUG:urllib3.connectionpool:https://npclassifier.ucsd.edu:443 "GET /classify?smiles=CC%5BC@H%5D(C)%5BC@@H%5D(C(=O)N%5BC@@H%5D(%5BC@@H%5D(C)O)C(=O)N%5BC@@H%5D(CC(=O)O)C(=O)O)NC(=O)%5BC@H%5D(CCCCN)NC(=O)%5BC@H%5D(%5BC@@H%5D(C)O)NC(=O)%5BC@H%5D(CCC(=O)N)NC(=O)%5BC@H%5D(%5BC@@H%5D(C)CC)NC(=O)%5BC@H%5D(CC(C)C)NC(=O)%5BC@H%5D(CC1=CNC2=CC=CC=C21)NC(=O)%5BC@H%5D(CC(=O)N)NC(=O)%5BC@H%5D(%5BC@@H%5D(C)CC)NC(=O)%5BC@H%5D(CC3=CC=CC=C3)NC(=O)%5BC@H%5D(CC(=O)O)NC(=O)%5BC@H%5D(CCCNC(=N)N)NC(=O)%5BC@H%5D(C)NC(=O)%5BC@H%5D(C)NC(=O)%5BC@H%5D(CC(C)C)NC(=O)%5BC@H%5D(CC(=O)N)NC(=O)%5BC@H%5D(CC(=O)O)NC(=O)%5BC@H%5D(CC(C)C)NC(=O)%5BC@H%5D(%5BC@@H%5D(C)CC)NC(=O)%5BC@H%5D(%5BC@@H%5D(C)O)NC(=O)%5BC@H%5D(CC(=O)N)NC(=O)%5BC@H%5D(CCSC)NC(=O)%5BC@H%5D(CCC(=O)O)NC(=O)%5BC@H%5D(CC(=O)O)NC(=O)%5BC@H%5D(CO)NC(=O)%5BC@H%5D(CC4=CC=CC=C4)NC(=O)%5BC@H%5D(CO)NC(=O)CNC(=O)%5BC@H%5D(CC(=O)O)NC(=O)CNC(=O)%5BC@H%5D(CC5=CN=CN5)N HTTP/1.1" 200 540


  9%|▊         | 114/1314 [00:19<03:07,  6.41it/s]DEBUG:urllib3.connectionpool:Resetting dropped connection: npclassifier.ucsd.edu
DEBUG:urllib3.connectionpool:https://npclassifier.ucsd.edu:443 "GET /classify?smiles=CC1=C(C=CC2=C1N=C(C=C2O%5BC@@H%5D3C%5BC@@H%5D4%5BC@@H%5D(C3)C(=O)N(CCCC/C=C%5C%5BC@@H%5D5C%5BC@%5D5(NC4=O)C(=O)NS(=O)(=O)C6CC6)C)C7=NC(=CS7)C(C)C)OC HTTP/1.1" 200 407
 11%|█▏        | 150/1314 [00:26<03:34,  5.42it/s]DEBUG:urllib3.connectionpool:https://npclassifier.ucsd.edu:443 "GET /classify?smiles=CCN%5BC@@H%5D1C%5BC@@H%5D(%5BC@H%5D(%5BC@@H%5D(%5BC@H%5D1O%5BC@@H%5D2%5BC@@H%5D(%5BC@H%5D(%5BC@@%5D(CO2)(C)O)NC)O)O)O%5BC@@H%5D3%5BC@@H%5D(CC=C(O3)CN)N)N.CCN%5BC@@H%5D1C%5BC@@H%5D(%5BC@H%5D(%5BC@@H%5D(%5BC@H%5D1O%5BC@@H%5D2%5BC@@H%5D(%5BC@H%5D(%5BC@@%5D(CO2)(C)O)NC)O)O)O%5BC@@H%5D3%5BC@@H%5D(CC=C(O3)CN)N)N.OS(=O)(=O)O.OS(=O)(=O)O.OS(=O)(=O)O.OS(=O)(=O)O.OS(=O)(=O)O HTTP/1.1" 200 391


 15%|█▍        | 193/1314 [00:32<03:04,  6.08it/s]DEBUG:urllib3.connectionpool:https://npclassifier.ucsd.edu:443 "GET /classify?smiles=C1CC(=O)NC2=C1C=CC(=C2)OCCCCN3CCN(CC3)C4=C(C(=CC=C4)Cl)Cl HTTP/1.1" 200 340


...

 89%|████████▉ | 1167/1314 [03:26<00:25,  5.81it/s]DEBUG:urllib3.connectionpool:Resetting dropped connection: npclassifier.ucsd.edu
DEBUG:urllib3.connectionpool:https://npclassifier.ucsd.edu:443 "GET /classify?smiles=CC1=CC=C(C=C1)C2=C(N3C=C(C=CC3=N2)C)CC(=O)N(C)C.CC1=CC=C(C=C1)C2=C(N3C=C(C=CC3=N2)C)CC(=O)N(C)C.C(C(C(=O)O)O)(C(=O)O)O HTTP/1.1" 200 319
 90%|█████████ | 1185/1314 [03:29<00:22,  5.76it/s]DEBUG:urllib3.connectionpool:https://npclassifier.ucsd.edu:443 "GET /classify?smiles=C1C2=CC=CC=C2N(C3=CC=CC=C3C1=O)C(=O)N HTTP/1.1" 200 289


 91%|█████████▏| 1202/1314 [03:31<00:10, 10.85it/s]DEBUG:urllib3.connectionpool:https://npclassifier.ucsd.edu:443 "GET /classify?smiles=CCOC(=O)%5BC@H%5D(CCC1=CC=CC=C1)N%5BC@@H%5D(C)C(=O)N2CC3=CC=CC=C3C%5BC@H%5D2C(=O)O HTTP/1.1" 200 356
DEBUG:urllib3.connectionpool:https://npclassifier.ucsd.edu:443 "GET /classify?smiles=CN1CCN2C(C1)C3=CC=CC=C3CC4=C2N=CC=C4 HTTP/1.1" 200 300
 93%|█████████▎| 1225/1314 [03:35<00:16,  5.45it/s]DEBUG:urllib3.connectionpool:https://npclassifier.ucsd.edu:443 "GET /classify?smiles=CN1CCN(CC1)/C=C%5C2/C(=O)N3C(=N2)CN=C(C4=C3C=CC(=C4)%5BN+%5D(=O)%5BO-%5D)C5=CC=CC=C5Cl.CS(=O)(=O)O HTTP/1.1" 500 290


 96%|█████████▌| 1262/1314 [03:42<00:13,  3.84it/s]DEBUG:urllib3.connectionpool:https://npclassifier.ucsd.edu:443 "GET /classify?smiles=C%5BC@H%5D(C1=CC(=CC(=C1)C(F)(F)F)C(F)(F)F)O%5BC@@H%5D2%5BC@@H%5D(N(CCO2)CC3=NNC(=O)N3)C4=CC=C(C=C4)F HTTP/1.1" 200 340


 98%|█████████▊| 1287/1314 [03:47<00:04,  5.57it/s]DEBUG:urllib3.connectionpool:Resetting dropped connection: npclassifier.ucsd.edu
DEBUG:urllib3.connectionpool:https://npclassifier.ucsd.edu:443 "GET /classify?smiles=COC1=CC(=CC(=C1Br)OC)CC2=CN=C(N=C2N)N HTTP/1.1" 200 286
 99%|█████████▉| 1304/1314 [03:50<00:01,  5.64it/s]DEBUG:urllib3.connectionpool:https://npclassifier.ucsd.edu:443 "GET /classify?smiles=CC%5BC@@%5D1(C2=C(COC1=O)C(=O)N3CC4=C(C5=CC=CC=C5N=C4C3=C2)CCNC(C)C)O.Cl HTTP/1.1" 200 153


100%|██████████| 1314/1314 [03:52<00:00,  5.64it/s]


ClassyFire
Ping API

In [13]:
resolved_ik_number_list = [0, 0]
total_inchi_number = len(all_inchikeys)

while True:
    
    start_time = time.time()
    
    print('%s inchikey to resolve' % total_inchi_number )
    get_classifications_cf_mod(all_inchikeys, par_level= 50)
    
    cleanse('all_json.json', 'all_json.json')
    
    with open("all_json.json") as tweetfile:
        jsondic = json.loads(tweetfile.read())

    df = json_normalize(jsondic)
    df = df.drop_duplicates('smiles')
    resolved_ik_number = len( df.drop_duplicates('smiles').inchikey )
    resolved_ik_number_list.append( resolved_ik_number )
    print('%s resolved smiles' % resolved_ik_number )
    print("done in --- %s seconds ---" % (time.time() - start_time))
    
    if resolved_ik_number_list[-1] < resolved_ik_number_list[-2] or resolved_ik_number_list[-1] == resolved_ik_number_list[-3]:
        break

1314 inchikey to resolve
1166 resolved smiles
done in --- 29.414042949676514 seconds ---
1314 inchikey to resolve
1166 resolved smiles
done in --- 3.7770962715148926 seconds ---
1314 inchikey to resolve
1166 resolved smiles
done in --- 3.9432308673858643 seconds ---


In [14]:
df.head()

,smiles,inchikey,intermediate_nodes,alternative_parents,molecular_framework,substituents,description,external_descriptors,ancestors,predicted_chebi_terms,...,class.url,subclass.name,subclass.description,subclass.chemont_id,subclass.url,direct_parent.name,direct_parent.description,direct_parent.chemont_id,direct_parent.url,identifier
0,CC(C)C(=O)NC1=CC(=C(C=C1)[N+]([O-])=O)C(F)(F)F,InChIKey=MKXKFYHWDHIYRV-UHFFFAOYSA-N,[],"[{'name': 'Nitrobenzenes', 'description': 'Com...",Aromatic homomonocyclic compounds,"[Trifluoromethylbenzene, Nitrobenzene, Anilide...",This compound belongs to the class of organic ...,"[{'source': 'CHEBI', 'source_id': 'CHEBI:5132'...","[Alkyl fluorides, Alkyl halides, Allyl-type 1,...","[nitrobenzenes (CHEBI:48109), anilide (CHEBI:1...",...,http://classyfire.wishartlab.com/tax_nodes/C00...,Trifluoromethylbenzenes,Organofluorine compounds that contain a benzen...,CHEMONTID:0004551,http://classyfire.wishartlab.com/tax_nodes/C00...,Trifluoromethylbenzenes,Organofluorine compounds that contain a benzen...,CHEMONTID:0004551,http://classyfire.wishartlab.com/tax_nodes/C00...,NaN
1,OC(=O)C(CC1=CC(O)=NC2=CC=CC=C12)NC(=O)C1=CC=C(...,InChIKey=ALLWOAVDORUJLA-UHFFFAOYSA-N,"[{'name': 'Benzamides', 'description': 'Organi...","[{'name': 'N-acyl-alpha amino acids', 'descrip...",Aromatic heteropolycyclic compounds,"[N-acyl-alpha amino acid or derivatives, N-acy...",This compound belongs to the class of organic ...,[],"[4-halobenzoic acids and derivatives, Alpha am...","[N-acyl-amino acid (CHEBI:51569), quinolone (C...",...,http://classyfire.wishartlab.com/tax_nodes/C00...,Benzoic acids and derivatives,Organic compounds containing a carboxylic acid...,CHEMONTID:0000176,http://classyfire.wishartlab.com/tax_nodes/C00...,Hippuric acids,"Compounds containing hippuric acid, which cons...",CHEMONTID:0001318,http://classyfire.wishartlab.com/tax_nodes/C00...,NaN
2,COC(=O)C1=C(C)NC(C)=C(C1C1=CC(=CC=C1)[N+]([O-]...,InChIKey=ZDXUKAKRHYTAKV-UHFFFAOYSA-N,[],"[{'name': 'Nitrobenzenes', 'description': 'Com...",Aromatic heteromonocyclic compounds,"[Diphenylmethane, Dihydropyridinecarboxylic ac...",This compound belongs to the class of organic ...,[],"[Allyl-type 1,3-dipolar organic compounds, Alp...","[nitrobenzenes (CHEBI:48109), dihydropyridine ...",...,http://classyfire.wishartlab.com/tax_nodes/C00...,Diphenylmethanes,"Compounds containing a diphenylmethane moiety,...",CHEMONTID:0000369,http://classyfire.wishartlab.com/tax_nodes/C00...,Diphenylmethanes,"Compounds containing a diphenylmethane moiety,...",CHEMONTID:0000369,http://classyfire.wishartlab.com/tax_nodes/C00...,NaN
3,COC1=CC2=NC=CC(OC3=CC(Cl)=C(NC(=O)NC4=NOC(C)=C...,InChIKey=SPMVMDHWKHCIDT-UHFFFAOYSA-N,[],"[{'name': 'Quinolines and derivatives', 'descr...",Aromatic heteropolycyclic compounds,"[Diaryl ether, N-phenylurea, Quinoline, Phenox...",This compound belongs to the class of organic ...,[],"[Alkyl aryl ethers, Anisoles, Aryl chlorides, ...","[quinolines (CHEBI:26513), ureas (CHEBI:47857)...",...,http://classyfire.wishartlab.com/tax_nodes/C00...,Ethers,Compounds bearing an ether group with the form...,CHEMONTID:0000254,http://classyfire.wishartlab.com/tax_nodes/C00...,Diarylethers,Organic compounds containing the dialkyl ether...,CHEMONTID:0002463,http://classyfire.wishartlab.com/tax_nodes/C00...,NaN
4,CN1C2=C(OC(=O)N(C2=O)C2=CC=CC=N2)C2=CC=CC=C2S1...,InChIKey=OEHFRZLKGRKFAS-UHFFFAOYSA-N,[],"[{'name': 'Pyridines and derivatives', 'descri...",Aromatic heteropolycyclic compounds,"[Benzothiazine, Pyridine, Benzenoid, Organosul...",This compound belongs to the class of organic ...,"[{'source': 'CHEBI', 'source_id': 'CHEBI:76133...","[Azacyclic compounds, Benzenoids, Benzothiazin...","[pyridines (CHEBI:26421), sulfonamide (CHEBI:3...",...,http://classyfire.wishartlab.com/tax_nodes/C00...,NaN,NaN,NaN,NaN,Benzothiazines,Organic compounds containing a benzene fused t...,CHEMONTID:0000309,http://classyfire.wishartlab.com/tax_nodes/C00...,NaN


In [15]:
df.to_csv('classyfire_smiles.tsv.gz', sep='\t', compression='gzip')

In [17]:
np_classifier_df.to_csv('npclassifier_smiles.tsv.gz', sep='\t', compression='gzip')

In [18]:
np_classifier_df

,smiles,class_results,superclass_results,pathway_results
0,COC1=CC=C(C=C1)CCN2CCC(CC2)NC3=NC4=CC=CC=C4N3C...,[],[],[Alkaloids]
1,CCOC(=O)[C@H](CCC1=CC=CC=C1)N[C@@H](C)C(=O)N2C...,[Dipeptides],[Small peptides],[Amino acids and Peptides]
2,CCCCCCCN(CC)CCCC(C1=CC=C(C=C1)NS(=O)(=O)C)O.CC...,[],[],[]
3,COC(=O)[C@H](C1=CC=CC=C1Cl)N2CCC3=C(C2)C=CS3.O...,[],[],[]
4,CC1=CC=CC=C1C(=O)NC2=CC=C(C=C2)C(=O)N3CCCC(C4=...,[],[],[]
...,...,...,...,...
1088,CC1=C2[C@H](C(=O)[C@@]3([C@H](C[C@@H]4[C@]([C@...,"[Taxane diterpenoids, Tetracyclic diterpenoids]",[Diterpenoids],[Terpenoids]
1089,C1=NC(=NC(=O)N1[C@H]2[C@@H]([C@@H]([C@H](O2)CO...,[Purine nucleos(t)ides],[Nucleosides],[Carbohydrates]
1090,CCNC(=NCCCC[C@H](C(=O)N[C@@H](CC(C)C)C(=O)N[C@...,[Linear peptides],[Oligopeptides],[Amino acids and Peptides]
1091,CC(C)NCC(COC1=CC=C(C=C1)CCOCC2CC2)O,[],[],[Alkaloids]
